In [82]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from sqlalchemy import create_engine
import os


# OAuth token request
url = "https://accounts.zoho.com/oauth/v2/token"
params = {
    "client_id": "1000.8KP5FVR2CASRD70V5D6EN3EHNZXC0W",
    "client_secret": "2e76b56ef1a4ff32d4ae87f35024be17da3e641f9e",
    "grant_type": "client_credentials",
    "scope": "ZohoAnalytics.data.read",
    "soid": "ZohoCRM.696664214"
}

response = requests.post(url, params=params)
if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Token obtained successfully")
else:
    print(f"Failed to obtain token. Status code: {response.status_code}")
    exit()

# Make the API request
url = "https://analyticsapi.zoho.com/api/ashutosh@raptorsupplies.com/Zoho%20CRM%20Analytics/Logistic 2?ZOHO_ACTION=EXPORT&ZOHO_OUTPUT_FORMAT=XML&ZOHO_ERROR_FORMAT=XML&ZOHO_API_VERSION=1.0"

headers = {
    "Authorization": f"Zoho-oauthtoken {access_token}"
}

response = requests.get(url, headers=headers)

# Check the response status
if response.status_code == 200:
    # Parse the XML from response.content
    root = ET.fromstring(response.content)

    # Initialize a list to hold row data
    rows_data = []

    # Extract data from each row
    for row in root.findall('.//row'):
        # Create a dictionary for each row's data
        row_data = {}
        for column in row:
            column_name = column.attrib.get('name')  # Get the 'name' attribute as column name
            row_data[column_name] = column.text  # Get the text content of the column
        rows_data.append(row_data)

    # Create a DataFrame from the list of rows
df = pd.DataFrame(rows_data)
print(df)

Token obtained successfully
       SNo      Version Sheet.Stage  Date tracking Enter      Invoice  \
0    46794                  TRACK 2  01/03/2024 14:46:27  UK19670|OAU   
1    46993  CANCELLED RAPTOR ORDERS  02/03/2024 17:17:21  UK20004|QCD   
2    47386                  TRACK 3  07/03/2024 04:31:08  NL10750|QCD   
3    47393  CANCELLED RAPTOR ORDERS  07/03/2024 12:15:13  UK20217|QCD   
4    47394                  TRACK 1  07/03/2024 12:20:02  UK18528|QPX   
..     ...                      ...                  ...          ...   
943  60976                  TRACK 1  24/06/2024 18:26:25  NL12344|QCX   
944  60989                  TRACK 1  24/06/2024 19:25:02  UK23173|QAX   
945  60999                  TRACK 1  24/06/2024 19:57:01  UK23401|QCD   
946  61008                  TRACK 1  24/06/2024 23:56:47  UK23385|OAD   
947  61010                  TRACK 1  24/06/2024 23:58:43   AU4575|OAD   

              Supplier Name  \
0                KSC DIRECT   
1                      ebay   
2 

In [83]:
df=df[df['Version Sheet.Stage']!='ORDER CLOSED']

In [84]:
df[['Pic', 'Item Pics', 'Weight_LBS', 'Dimensions_Inches', 'Remark by Robert', 'Remark by Logistic team', 'Delivery Date']] = None
cols = list(df.columns)
tracking_index = cols.index('Tracking Number')
cols.insert(tracking_index + 1, cols.pop(cols.index('Delivery Date')))
df = df[cols]

In [85]:
# df=df.drop(['Version Sheet.Date of Funds Received','Version Sheet.Expected Shipment Date'],axis=1)

In [86]:
df['SNo']=df.SNo.astype(int)

In [87]:
df['Purchase Cost'] = df['Purchase Cost'].astype(float)
df['Pic'] = df['Purchase Cost'].apply(lambda x: 'Yes' if x > 50 else 'No')
df['Item Pics'] = df['Purchase Cost'].apply(lambda x: 'Yes' if x > 1500 else 'No')
df['Purchase Cost'] = df['Purchase Cost'].astype(str)


In [88]:
df.loc[df['Version Sheet.Placed the Order with Supplier'] == 'Multiple Vendors', 'Remark by Logistic team'] = df['Remark by Logistic team'].fillna('') + 'Multiple line item'


In [89]:
df['Date tracking Enter'] = pd.to_datetime(df['Date tracking Enter'], format='%d/%m/%Y %H:%M:%S')

# Sort the DataFrame by 'Date tracking Enter' in ascending order
df_sorted = df.sort_values(by='Date tracking Enter', ascending=True)

# Extract the earliest instance of each unique 'Invoice'
earliest_invoices = df_sorted.drop_duplicates(subset='Invoice', keep='first')

# Extract the remaining rows with 'Invoice' values
remaining_invoices = df_sorted[df_sorted.duplicated(subset='Invoice', keep='first')]

# Create an empty DataFrame to store the final sorted result
df_final = pd.DataFrame(columns=df.columns)

# Append remaining invoices just below their respective earliest invoices
for invoice in earliest_invoices['Invoice']:
    # Append the earliest invoice row
    df_final = pd.concat([df_final, earliest_invoices[earliest_invoices['Invoice'] == invoice]])
    # Append the remaining invoice rows
    df_final = pd.concat([df_final, remaining_invoices[remaining_invoices['Invoice'] == invoice]])

# Append rows without 'Invoice' values (if needed)
df_no_invoices = df_sorted[df_sorted['Invoice'].isna()]
df_final = pd.concat([df_final, df_no_invoices])

# Reset index for the final DataFrame
df_final = df_final.reset_index(drop=True)

# Display the final sorted DataFrame
print(df_final)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_4876\1178887668.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_final, earliest_invoices[earliest_invoices['Invoice'] == invoice]])


       SNo      Version Sheet.Stage Date tracking Enter      Invoice  \
0    46794                  TRACK 2 2024-03-01 14:46:27  UK19670|OAU   
1    46993  CANCELLED RAPTOR ORDERS 2024-03-02 17:17:21  UK20004|QCD   
2    47386                  TRACK 3 2024-03-07 04:31:08  NL10750|QCD   
3    50573                  TRACK 3 2024-04-05 09:39:51  NL10750|QCD   
4    60182                  TRACK 3 2024-06-18 19:11:34  NL10750|QCD   
..     ...                      ...                 ...          ...   
943  60969                  TRACK 1 2024-06-24 17:49:32  UK23302|OAD   
944  60989                  TRACK 1 2024-06-24 19:25:02  UK23173|QAX   
945  60999                  TRACK 1 2024-06-24 19:57:01  UK23401|QCD   
946  61008                  TRACK 1 2024-06-24 23:56:47  UK23385|OAD   
947  61010                  TRACK 1 2024-06-24 23:58:43   AU4575|OAD   

          Supplier Name Supplier (Grainger / Non-Grainger)  \
0            KSC DIRECT                         KSC DIRECT   
1          

In [90]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Function to authenticate and get the Google Sheets client
def authenticate_google_sheets(json_credentials_path):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(json_credentials_path, scope)
    client = gspread.authorize(creds)
    return client

# Function to fetch data from a worksheet and convert it to a DataFrame
def fetch_worksheet_to_dataframe(sheet_name, worksheet_name, json_credentials_path):
    try:
        # Authenticate and get the Google Sheets client
        client = authenticate_google_sheets(json_credentials_path)
        print("Authenticated successfully")

        # Open the existing Google Sheet by name
        sheet = client.open(sheet_name)
        print(f"Opened Google Sheet: {sheet_name}")

        # Get the worksheet by name
        worksheet = sheet.worksheet(worksheet_name)
        print(f"Accessing worksheet: {worksheet_name}")

        # Fetch all data from the worksheet
        data = worksheet.get_all_values()
        print("Fetched data from worksheet")

        # Convert the data to a DataFrame
        df = pd.DataFrame(data[1:], columns=data[0])
        print("Converted data to DataFrame")

        return df

    except gspread.exceptions.APIError as api_error:
        print(f"Google Sheets API error: {api_error}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Example usage
if __name__ == "__main__":
    # Path to the JSON credentials file
    json_credentials_path = 'divine-arcade-406611-e0729e40870d.json'

    # Name of the existing Google Sheet
    sheet_name = 'Tracking Sheet for Charlotte.xlsx'

    # Name of the worksheet
    worksheet_name = 'Test'

    # Fetch data from the worksheet into a DataFrame
    df_old = fetch_worksheet_to_dataframe(sheet_name, worksheet_name, json_credentials_path)

    if df_old is not None:
        print("Data fetched successfully")
        print(df_old.head())
    else:
        print("Failed to fetch data")

Authenticated successfully
Opened Google Sheet: Tracking Sheet for Charlotte.xlsx
Accessing worksheet: Test
Fetched data from worksheet
Converted data to DataFrame
Data fetched successfully
     SNo      Version Sheet.Stage  Date tracking Enter      Invoice  \
0  46794                  TRACK 2  2024-03-01 14:46:27  UK19670|OAU   
1  46993  CANCELLED RAPTOR ORDERS  2024-03-02 17:17:21  UK20004|QCD   
2  47386                  TRACK 3  2024-03-07 04:31:08  NL10750|QCD   
3  50573                  TRACK 3  2024-04-05 09:39:51  NL10750|QCD   
4  60182                  TRACK 3  2024-06-18 19:11:34  NL10750|QCD   

  Supplier Name Supplier (Grainger / Non-Grainger)  \
0    KSC DIRECT                         KSC DIRECT   
1          ebay                  ( cancelled) ebay   
2     indsencon        indsencon + andersonprocess   
3      anderson        indsencon + andersonprocess   
4     indsencon        indsencon + andersonprocess   

  Version Sheet.Placed the Order with Supplier         Tra

In [91]:
df_old

,SNo,Version Sheet.Stage,Date tracking Enter,Invoice,Supplier Name,Supplier (Grainger / Non-Grainger),Version Sheet.Placed the Order with Supplier,Tracking Number,Delivery Date,Tracking Courier Details.Courier API List,...,Version Sheet.Order Payment Received Status,Version Sheet.Date of Funds Received,Version Sheet.Expected Shipment Date,Version Sheet.Date of Order Received,Pic,Item Pics,Weight_LBS,Dimensions_Inches,Remark by Robert,Remark by Logistic team
0,46794,TRACK 2,2024-03-01 14:46:27,UK19670|OAU,KSC DIRECT,KSC DIRECT,Yes,1ZRF58610394781994,,UPS,...,PAID,15/02/2024,Flat Shipping (5 - 7 days),15/02/2024,Yes,No,,,,
1,46993,CANCELLED RAPTOR ORDERS,2024-03-02 17:17:21,UK20004|QCD,ebay,( cancelled) ebay,Yes,9405508205499844047803,,USPS,...,NOT PAID,,2-3 WEEKS,28/02/2024,No,No,,,,
2,47386,TRACK 3,2024-03-07 04:31:08,NL10750|QCD,indsencon,indsencon + andersonprocess,Multiple Vendors,9434611206204970878331,,USPS,...,PAID,18/06/2024,6-8 WEEKS,01/03/2024,Yes,No,,,,
3,50573,TRACK 3,2024-04-05 09:39:51,NL10750|QCD,anderson,indsencon + andersonprocess,Multiple Vendors,1z4e93640372781255,,UPS,...,PAID,18/06/2024,6-8 WEEKS,01/03/2024,Yes,No,,,,
4,60182,TRACK 3,2024-06-18 19:11:34,NL10750|QCD,indsencon,indsencon + andersonprocess,Multiple Vendors,9434611206204137800311,,USPS,...,PAID,18/06/2024,6-8 WEEKS,01/03/2024,Yes,No,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,60969,TRACK 1,2024-06-24 17:49:32,UK23302|OAD,buyenerpac,buyenerpac,Yes,1Z51WY390351926335,,UPS,...,PAID,19/06/2024,Flat Rate Shipping (5 - 7 days),19/06/2024,Yes,No,,,,
957,60989,TRACK 1,2024-06-24 19:25:02,UK23173|QAX,hvacandtoolsdirect,hvacandtoolsdirect,Yes,276258131600,,FEDEX,...,PAID,17/06/2024,1-2 WEEKS,14/06/2024,Yes,No,,,,
958,60999,TRACK 1,2024-06-24 19:57:01,UK23401|QCD,thedrainagesource,thedrainagesource,Yes,1ZEW21850312049873,,UPS,...,NOT PAID,,5-7 DAYS,21/06/2024,Yes,No,,,,
959,61008,TRACK 1,2024-06-24 23:56:47,UK23385|OAD,toolup,toolup,Yes,1Z8723500321347454,,UPS,...,PAID,21/06/2024,Express (5 - 7 days),20/06/2024,No,No,,,,


In [92]:
import pandas as pd
from datetime import datetime, timedelta

# Ensure 'Date tracking Enter' is in datetime format
df_old['Date tracking Enter'] = pd.to_datetime(df_old['Date tracking Enter'])

# Get today's date
today = datetime.today()

# Calculate the date 7 days ago
seven_days_ago = today - timedelta(days=7)

# Filter the DataFrame for records from the last 7 days
df_last_7_days = df_old[df_old['Date tracking Enter'] >= seven_days_ago]

# Display the filtered DataFrame
df_last_7_days

,SNo,Version Sheet.Stage,Date tracking Enter,Invoice,Supplier Name,Supplier (Grainger / Non-Grainger),Version Sheet.Placed the Order with Supplier,Tracking Number,Delivery Date,Tracking Courier Details.Courier API List,...,Version Sheet.Order Payment Received Status,Version Sheet.Date of Funds Received,Version Sheet.Expected Shipment Date,Version Sheet.Date of Order Received,Pic,Item Pics,Weight_LBS,Dimensions_Inches,Remark by Robert,Remark by Logistic team
4,60182,TRACK 3,2024-06-18 19:11:34,NL10750|QCD,indsencon,indsencon + andersonprocess,Multiple Vendors,9434611206204137800311,,USPS,...,PAID,18/06/2024,6-8 WEEKS,01/03/2024,Yes,No,,,,
21,60803,TRACK 1,2024-06-21 20:02:10,UK20279|QCD,MCM PO UK20279(4),MCM,Multiple Vendors,1Z0835200377901977,,UPS,...,NOT PAID,,7-10 DAYS,08/03/2024,Yes,No,,,,
32,60340,TRACK 2,2024-06-19 16:36:37,UK17236|QAX,KSC Direct,KSC Direct,Yes,1Z4560160326196011,,UPS,...,PAID,22/11/2023,09-11 Weeks,30/10/2023,Yes,Yes,,,,
64,60687,TRACK 1,2024-06-21 15:55:11,UK20441|QCD,Grainger,zoro ( 2 units ) + grainger ( 15 units) ( qou...,Multiple Vendors,403789183621,24-06-24,FEDEX,...,NOT PAID,,5-8 WEEKS,14/03/2024,No,No,,,,
71,60223,TRACK 1,2024-06-19 12:35:03,UK15520|QAD,mro,MRO \nQUOTE,Yes,745625052848,17-06-24,FEDEX,...,PAID,17/08/2023,08-10 WEEKS,16/08/2023,Yes,No,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,60969,TRACK 1,2024-06-24 17:49:32,UK23302|OAD,buyenerpac,buyenerpac,Yes,1Z51WY390351926335,,UPS,...,PAID,19/06/2024,Flat Rate Shipping (5 - 7 days),19/06/2024,Yes,No,,,,
957,60989,TRACK 1,2024-06-24 19:25:02,UK23173|QAX,hvacandtoolsdirect,hvacandtoolsdirect,Yes,276258131600,,FEDEX,...,PAID,17/06/2024,1-2 WEEKS,14/06/2024,Yes,No,,,,
958,60999,TRACK 1,2024-06-24 19:57:01,UK23401|QCD,thedrainagesource,thedrainagesource,Yes,1ZEW21850312049873,,UPS,...,NOT PAID,,5-7 DAYS,21/06/2024,Yes,No,,,,
959,61008,TRACK 1,2024-06-24 23:56:47,UK23385|OAD,toolup,toolup,Yes,1Z8723500321347454,,UPS,...,PAID,21/06/2024,Express (5 - 7 days),20/06/2024,No,No,,,,


In [93]:
new_rows_df=df_last_7_days

In [94]:
new_rows_df=new_rows_df[new_rows_df['Delivery Date']=='']

In [95]:
new_rows_df_1=new_rows_df[new_rows_df['Tracking Courier Details.Courier  API List']=='FEDEX']

In [96]:
new_rows_df_1

,SNo,Version Sheet.Stage,Date tracking Enter,Invoice,Supplier Name,Supplier (Grainger / Non-Grainger),Version Sheet.Placed the Order with Supplier,Tracking Number,Delivery Date,Tracking Courier Details.Courier API List,...,Version Sheet.Order Payment Received Status,Version Sheet.Date of Funds Received,Version Sheet.Expected Shipment Date,Version Sheet.Date of Order Received,Pic,Item Pics,Weight_LBS,Dimensions_Inches,Remark by Robert,Remark by Logistic team
482,60816,TRACK 1,2024-06-22 02:28:35,UK21890/US0462|QCD,Fotronic Corporation,Grainger + Fotronic (Quote) {04line item},Multiple Vendors,776989912468,,FEDEX,...,NOT PAID,,12-16 Weeks,06/05/2024,No,No,,,,
526,60062,PARTIALLY SHIPPED FROM SUPPLIER,2024-06-18 14:45:12,NL12344|QCX,webstaurantstore,webstaurantstore\n\nPlease note @Shivansh Shek...,Yes,776891023115 ; 776891024589,,FEDEX,...,NOT PAID,,9-10 WEEKS,03/06/2024,Yes,Yes,,,,
771,60224,TRACK 2,2024-06-19 12:42:31,NL12064|QCD,MRO,MRO SUPPLY\n\nQUOTE ATTACHED,Yes,709796880067 ;709796880078,,FEDEX,...,NOT PAID,,3-4 WEEKS,20/05/2024,Yes,Yes,,,,
781,60257,CANCELLED RAPTOR ORDERS,2024-06-19 13:43:42,NL11593|OAD,GRAINGER,Grainger 463D96: bring this to CLT then ship v...,Yes,4041019368,,FEDEX,...,PAID,22/04/2024,Flat Shipping (5 - 7 days),22/04/2024,Yes,No,,,,
843,60508,TRACK 1,2024-06-20 17:10:27,NL12448|QCD,Grainger,grainger,Yes,400333125269 ; 400333126059,,FEDEX,...,NOT PAID,,5-8 DAYS,07/06/2024,Yes,No,,,,
848,60725,TRACK 1,2024-06-21 17:03:55,NL12448|QCD,Grainger,grainger,Yes,747131791003,,FEDEX,...,NOT PAID,,5-8 DAYS,07/06/2024,Yes,No,,,,
898,60665,TRACK 1,2024-06-21 15:10:17,NL12675|QCD,Ebay Order number: 01-11726-57196,ebay,Multiple Vendors,1Z0109FX0397819881,,FEDEX,...,NOT PAID,,5-7 DAYS,19/06/2024,No,No,,,,
908,60720,TRACK 2,2024-06-21 17:00:47,SG4472|QAD,Grainger,"grainger\nDDP SG, Air freight (GO SHIPPO (2 SH...",Yes,403789213570 ; 403789214223,,FEDEX,...,PAID,20/06/2024,7-11 DAYS,12/06/2024,Yes,Yes,,,,
911,60744,TRACK 1,2024-06-21 17:20:46,SG4225|QAD,,Ametek OEM,Yes,776983698880,,FEDEX,...,PAID,16/05/2024,02-03 Weeks,14/05/2024,Yes,No,,,,
927,60851,TRACK 1,2024-06-22 15:52:48,UK23362|QCD,astrotool,ASTROTOOL,Yes,741218079310,,FEDEX,...,NOT PAID,,08 days,20/06/2024,Yes,No,,,,


In [97]:
import requests
import json
import pandas as pd

# FedEx Production API credentials
CLIENT_ID = 'l7da4097b7bcfe4440ba7f860d7af9bf89'
CLIENT_SECRET = '900efa8a-0fdf-420e-9aaa-27efadea3bc3'

# Function to get access token
def get_access_token(client_id, client_secret):
    url = "https://apis.fedex.com/oauth/token"
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    }
    
    response = requests.post(url, headers=headers, data=data)
    
    if response.status_code == 200:
        token_info = response.json()
        access_token = token_info['access_token']
        return access_token
    else:
        print(f"Failed to get access token: {response.status_code}")
        print(response.text)
        return None

# Function to track a single shipment and fetch expected delivery date
def track_shipment(tracking_number, access_token):
    url = "https://apis.fedex.com/track/v1/trackingnumbers"
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}',
    }
    data = {
        "trackingInfo": [
            {
                "trackingNumberInfo": {
                    "trackingNumber": tracking_number
                }
            }
        ]
    }
    
    response = requests.post(url, headers=headers, json=data)
    
    if response.status_code == 200:
        tracking_info = response.json()
        print(f"Full response for tracking number {tracking_number}:\n", json.dumps(tracking_info, indent=2))  # Print full response for debugging
        
        try:
            complete_track_results = tracking_info['output']['completeTrackResults'][0]['trackResults'][0]
            
            # Check for estimated delivery date
            estimated_delivery = complete_track_results.get('estimatedDeliveryTimeWindow', {}).get('window', {}).get('ends')
            
            if estimated_delivery:
                return estimated_delivery
            else:
                # Fetch actual delivery date if estimated delivery date is not available
                for date_time in complete_track_results.get('dateAndTimes', []):
                    if date_time.get('type') == 'ACTUAL_DELIVERY':
                        return date_time.get('dateTime')
            return None
        except KeyError as e:
            print(f"Unexpected response structure or missing key: {e}")
            print(json.dumps(tracking_info, indent=2))  # Print full response for debugging
            return None
    else:
        print(f"Failed to track shipment: {response.status_code}")
        print(response.text)
        return None

# Example DataFrame
# data = {
#     'Tracking Number': ['730273739274', '362918040', '402458566008']
# }
# new_rows_df = pd.DataFrame(data)

# Add an empty 'Delivery Date' column
new_rows_df_1['Delivery Date'] = ''

# Print DataFrame columns to debug
print("DataFrame columns:", new_rows_df_1.columns)

# Step 1: Generate access token
access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)

# Step 2: Iterate over DataFrame and update 'Delivery Date' for all shipments
if access_token:
    for index, row in new_rows_df_1.iterrows():
        print(f"Processing row {index}: {row}")  # Debug statement to print row being processed
        tracking_numbers_str = row.get('Tracking Number')
        
        if tracking_numbers_str:
            tracking_numbers = tracking_numbers_str.split(';')
            dates = []
            
            for tracking_number in tracking_numbers:
                print(f"Tracking number: {tracking_number}")  # Debug statement to print tracking number
                delivery_date = track_shipment(tracking_number, access_token)
                if delivery_date:
                    dates.append(delivery_date)
                    print(f"Updated delivery date for tracking number {tracking_number}: {delivery_date}")  # Debug statement to print updated date
                else:
                    print(f"No delivery date found for tracking number {tracking_number}")
            
            new_rows_df_1.at[index, 'Delivery Date'] = ' | '.join(dates)

# Print the updated DataFrame
print(new_rows_df_1)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_4876\2221711404.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_rows_df_1['Delivery Date'] = ''


DataFrame columns: Index(['SNo', 'Version Sheet.Stage', 'Date tracking Enter', 'Invoice',
       'Supplier Name', 'Supplier (Grainger / Non-Grainger)',
       'Version Sheet.Placed the Order with Supplier', 'Tracking Number',
       'Delivery Date', 'Tracking Courier Details.Courier  API List',
       'Tracking Courier Details.Tracking Destination',
       'Version Sheet.Destination Point', 'Purchase Cost',
       'Version Sheet.Order Payment Received Status',
       'Version Sheet.Date of Funds Received',
       'Version Sheet.Expected Shipment Date',
       'Version Sheet.Date of Order Received', 'Pic', 'Item Pics',
       'Weight_LBS', 'Dimensions_Inches', 'Remark by Robert',
       'Remark by Logistic team'],
      dtype='object')
Processing row 482: SNo                                                                                  60816
Version Sheet.Stage                                                                TRACK 1
Date tracking Enter                                  

In [99]:
new_rows_df_1['Delivery Date'].value_counts()

Delivery Date
                                                         12
2024-06-21T12:09:48-04:00 | 2024-06-21T12:09:48-04:00     2
2024-06-19T12:12:18-04:00 | 2024-06-19T12:12:18-04:00     1
2024-06-17T11:47:11-04:00 | 2024-06-17T11:47:11-04:00     1
Name: count, dtype: int64

In [100]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Function to authenticate and get the Google Sheets client
def authenticate_google_sheets(json_credentials_path):
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name(json_credentials_path, scope)
    client = gspread.authorize(creds)
    return client

# Function to fetch data from a worksheet and convert it to a DataFrame
def fetch_worksheet_to_dataframe(sheet_name, worksheet_name, json_credentials_path):
    try:
        # Authenticate and get the Google Sheets client
        client = authenticate_google_sheets(json_credentials_path)
        print("Authenticated successfully")

        # Open the existing Google Sheet by name
        sheet = client.open(sheet_name)
        print(f"Opened Google Sheet: {sheet_name}")

        # Get the worksheet by name
        worksheet = sheet.worksheet(worksheet_name)
        print(f"Accessing worksheet: {worksheet_name}")

        # Fetch all data from the worksheet
        data = worksheet.get_all_values()
        print(f"Fetched data from worksheet: {data[:5]}")  # Print first 5 rows for debugging

        # Check if data is not empty
        if not data:
            print("No data found in the worksheet")
            return None, None

        # Convert the data to a DataFrame
        df_old = pd.DataFrame(data[1:], columns=data[0])
        print(f"Converted data to DataFrame: {df_old.head()}")

        return df_old, worksheet

    except gspread.exceptions.APIError as api_error:
        print(f"Google Sheets API error: {api_error}")
        return None, None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None, None

# Function to update the delivery dates in the Google Sheet
def update_delivery_dates(new_rows_df_1, df_old, worksheet):
    try:
        # Ensure only the necessary columns are used for update
        new_rows_df_1 = new_rows_df_1[['Tracking Number', 'Delivery Date']]
        print(f"new_rows_df_1: {new_rows_df_1.head()}")

        # Remove duplicates by taking the last occurrence of each Tracking Number
        new_rows_df_1 = new_rows_df_1.drop_duplicates(subset='Tracking Number', keep='last')

        # Set index for merging
        df_old.set_index('Tracking Number', inplace=True)
        new_rows_df_1.set_index('Tracking Number', inplace=True)

        # Update the 'Delivery Date' in the original DataFrame
        df_old.update(new_rows_df_1)
        df_old.reset_index(inplace=True)
        print(f"Updated df_old: {df_old.head()}")

        # Ensure 'Tracking Number' is just before 'Delivery Date'
        columns = df_old.columns.tolist()
        columns.remove('Tracking Number')
        columns.insert(columns.index('Delivery Date'), 'Tracking Number')
        df_old = df_old[columns]
        print(f"Reordered df_old: {df_old.head()}")

        # Convert the updated DataFrame back to a list of lists
        updated_data = [df_old.columns.values.tolist()] + df_old.values.tolist()

        # Clear the existing worksheet
        worksheet.clear()

        # Update the worksheet with the new data
        worksheet.update(updated_data)
        print("Delivery dates updated successfully")

    except Exception as e:
        print(f"An unexpected error occurred while updating delivery dates: {e}")

# Example usage
if __name__ == "__main__":
    # Path to the JSON credentials file
    json_credentials_path = 'divine-arcade-406611-e0729e40870d.json'

    # Name of the existing Google Sheet
    sheet_name = 'logistic_CLT'

    # Name of the worksheet
    worksheet_name = 'Test7'

    # Fetch data from the worksheet into a DataFrame
    df_old, worksheet = fetch_worksheet_to_dataframe(sheet_name, worksheet_name, json_credentials_path)

    if df_old is not None and worksheet is not None:
        print("Data fetched successfully from Google Sheets")
        print(df_old.head())  # Print the first few rows of df_old for debugging

        # Sample DataFrame with new delivery dates (Replace with your actual data)
        # new_rows_df = pd.DataFrame({
        #     'SNo': [1, 2, 3],
        #     'Version Sheet.Stage': ['Stage1', 'Stage2', 'Stage3'],
        #     'Date tracking Enter': ['2023-06-01', '2023-06-02', '2023-06-03'],
        #     'Invoice': ['Inv1', 'Inv2', 'Inv3'],
        #     'Supplier Name': ['Supplier1', 'Supplier2', 'Supplier3'],
        #     'Supplier (Grainger / Non-Grainger)': ['Grainger', 'Non-Grainger', 'Grainger'],
        #     'Version Sheet.Placed the Order with Supplier': ['Yes', 'No', 'Yes'],
        #     'Tracking Number': ['123', '456', '789'],
        #     'Delivery Date': ['2023-06-01', '2023-06-02', '2023-06-03'],
        #     'Tracking Courier Details.Courier  API List': ['API1', 'API2', 'API3'],
        #     'Tracking Courier Details.Tracking Destination': ['Dest1', 'Dest2', 'Dest3'],
        #     'Version Sheet.Destination Point': ['Point1', 'Point2', 'Point3'],
        #     'Purchase Cost': ['Cost1', 'Cost2', 'Cost3'],
        #     'Version Sheet.Order Payment Received Status': ['Received', 'Pending', 'Received'],
        #     'Version Sheet.Date of Funds Received': ['2023-06-01', '2023-06-02', '2023-06-03'],
        #     'Version Sheet.Expected Shipment Date': ['2023-06-01', '2023-06-02', '2023-06-03'],
        #     'Version Sheet.Date of Order Received': ['2023-06-01', '2023-06-02', '2023-06-03'],
        #     'Pic': ['Pic1', 'Pic2', 'Pic3'],
        #     'Item Pics': ['ItemPic1', 'ItemPic2', 'ItemPic3'],
        #     'Weight_LBS': ['10', '20', '30'],
        #     'Dimensions_Inches': ['10x10x10', '20x20x20', '30x30x30'],
        #     'Remark by Robert': ['Remark1', 'Remark2', 'Remark3'],
        #     'Remark by Logistic team': ['LogisticRemark1', 'LogisticRemark2', 'LogisticRemark3']
        # })

        # Update delivery dates in the Google Sheet
        update_delivery_dates(new_rows_df_1, df_old, worksheet)
    else:
        print("Failed to fetch data")


Authenticated successfully
Opened Google Sheet: logistic_CLT
Accessing worksheet: Test7
Fetched data from worksheet: [['SNo', 'Version Sheet.Stage', 'Date tracking Enter', 'Invoice', 'Supplier Name', 'Supplier (Grainger / Non-Grainger)', 'Version Sheet.Placed the Order with Supplier', 'Tracking Number', 'Delivery Date', 'Tracking Courier Details.Courier  API List', 'Tracking Courier Details.Tracking Destination', 'Version Sheet.Destination Point', 'Purchase Cost', 'Version Sheet.Order Payment Received Status', 'Version Sheet.Date of Funds Received', 'Version Sheet.Expected Shipment Date', 'Version Sheet.Date of Order Received', 'Pic', 'Item Pics', 'Weight_LBS', 'Dimensions_Inches', 'Remark by Robert', 'Remark by Logistic team'], ['46794', 'TRACK 2', '2024-03-01 14:46:27', 'UK19670|OAU', 'KSC DIRECT', 'KSC DIRECT', 'Yes', '1ZRF58610394781994', '', 'UPS', 'Track 1', 'DAP GERMANY', '128.63', 'PAID', '15/02/2024', 'Flat Shipping (5 - 7 days)', '15/02/2024', 'Yes', 'No', '', '', '', ''], ['